# 0 Preface
このノートブックではニューラルネットワークを用いて，以下の3つの基本的な問題に取り組みます．


*   CNNを使った画像分類

## 注意
このノートブックは，PyTorchの公式チュートリアルである [DEEP LEARNING WITH PYTORCH: A 60 MINUTE BLITZ](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html#deep-learning-with-pytorch-a-60-minute-blitz) の [Neural Networks](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py) を日本語に翻訳し，必要に応じて必要事項を適宜補足したものです．

# ニューラルネットワーク





ニューラルネットワークはtorch.nnパッケージを使って構築することができます．
前回のノートブックでAutoGradについて解説しましたが，torch.nnはAutoGradを用いたモデルを定義します．</br>

nn.Moduleはレイヤーと出力を返すforward(input)メソッドを含みます．

例として，以下の数字画像を分類するネットワークを見てください．（図は[PyTorch公式チュートリアル](https://pytorch.org/tutorials/)より）


<img src="https://pytorch.org/tutorials/_images/mnist.png" border="0"> 



これは非常にシンプルなフィードフォワードネットワークの例となっています．具体的には，画像を入力とし，いくつかのレイヤーを次々と伝播した後に，最終的にその画像のクラスを表すone-hotベクトルを出力します．

典型的なニューラルネットワークの訓練手順は以下の6ステップから成ります．

1. 訓練の対象である重み（パラメータ）をもつニューラルネットワークを定義する

2. データセットから入力データを繰り返しネットワークに入力する

3. 入力をネットワークに伝播させる

4. 最終的な出力が正解データとどれくらい異なるかを表す誤差を計算する

5. 誤差をネットワークの各重みに逆伝播させる（バックプロパゲーション）

6. 5.で得られた勾配をもとにネットワークの各重みを更新する（最も単純な更新式の例としては， `weight = weight - learning_rate * gradient` など）


【**注意**】 以降でdrive中にファイルを読み込めるよう以下を実行しておいてください．

In [1]:
from google.colab import drive # driveを接続
drive.mount('/content/drive')

Mounted at /content/drive


# ネットワークを定義する
では実際にニューラルネットワークを定義してみましょう．

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)   # 入力のチャネル数は1，出力のチャネル数は6，5x5の畳み込み層
        self.conv2 = nn.Conv2d(6, 16, 5)  # 入力のチャネル数は6，出力のチャネル数は16，5x5の畳み込み層
        
        # 線形層 y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)   
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))   # 大きさ(2,2)のMaxプーリング層
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)    # プーリングのWindowサイズの縦と横が同じ大きさなら，タプルではなく単一の値で指定できる
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]   # バッチの次元以外のすべての次元
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


ここで注目すべきは，backwardの計算（誤差逆伝播における重みの勾配の計算）は自分で実装する必要はないということです．forwardの計算のみを実装すれば，AutoGradによってbackwardの計算は裏側で自動的に組み込まれます．forwardの計算には，Tensorに対する演算を（基本的には）なんでも使うことができます．

学習対象となるモデルのパラメータは，net.parameters()で取得することができます．

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1の重みのSize

10
torch.Size([6, 1, 5, 5])


このネットワークに，大きさ32x32のランダム画像を入力してみます．


【**注意**】 MNISTデータセットの画像をこのネットワークに入力するためには，画像サイズを32x32にリサイズする必要があります．

In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1071,  0.1419,  0.1587, -0.0183, -0.0628, -0.0537,  0.1673,  0.0904,
         -0.0771,  0.0066]], grad_fn=<AddmmBackward>)


全ての重みの微分を0にリセットした後，ランダムな値でのlossの勾配をバックプロパゲーションします．

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

【**注意**】
- `torch.nn`はバッチのみをサポートします．`torch.nn`パッケージでは一貫して，1つのサンプルではなく，サンプルのミニバッチを入力として扱うというルールがあります．
- 例えば，`nn.Conv2d`では，バッチサイズ x チャネル数 x 縦 x 横 の4次元のTensorを入力として受け付けます． 
- もし1つのサンプルを用いたい場合は，`input.unsqueeze(0)`とすることで，バッチの次元を加えることで対処します．

先に進む前に，このあたりで今までの確認をしておきます．

- **`torch.Tensor`**<br/>
多次元配列のクラス．`backward()`などのAutoGradの操作をメソッドとしてもつ．また，各Tensorオブジェクトは，そのTensorに関する勾配として`grad`プロパティを保持します．

- **`nn.Module`**<br/>
ネットワークのモジュールのクラス．重みパラメータや各層の初期化，forward層とbackward層などをカプセル化しています．

- **`nn.Parameters`**<br/>
`nn.Module`に自動的に登録される重みパラメータのクラス．その実態はTensor．

# Loss Function
Loss Functionとは，(output, target)のペアを入力として，output（出力）とtarget（正解）がどれだけ異なるかを表す値（損失）を計算する関数のことです．

In [6]:
output = net(input)
target = torch.randn(10) 
target = target.view(1, -1)  # targetのshapeをoutputのshapeと同じにしておく
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0554, grad_fn=<MseLossBackward>)


ここで，.grad_fn 属性を使用してlossから各層を逆方向に追ってみると，次のような計算グラフが構成されていることがわかります．

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

`loss.backward()`を呼ぶと，lossに関してグラフ全体が微分され，グラフ中の`requires_grad=True`となっているTensorはその勾配を`.grad`プロパティに保持します．

# バックプロパゲーション
`loss.backward()`を呼ぶだけで，`loss`に関するバックプロパゲーションが行われます．今回のバックプロパゲーションを行う前に，以前のバックプロパゲーションで計算された勾配を0にリセットしておく必要があります．

以下では，`loss`に関するバックプロパゲーションを行い，`conv1`のバイアスパラメータの勾配の更新を確認しています．

In [7]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.0019,  0.0126, -0.0381, -0.0137, -0.0409, -0.0111])
conv1.bias.grad after backward
tensor([ 0.0080,  0.0232, -0.0357, -0.0154, -0.0341,  0.0130])


# 重みの更新
実用的な更新式として最も単純なものは**確率的勾配降下法（Stochastic Gradient Descent, SGD）**でしょう．SGDは，以下のような更新式で重みを更新する手法です．

```
weight = weight - learning_rate * gradient
```


これは次のように簡単なPythonコードで実装できます：

In [8]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

SGDの他にも，Nesterov-SGD, Adam, RMSPropなどの様々な重みの更新手法が存在し，PyTorchではそれらを`torch.optim`パッケージで管理しています．例えば，SGDを使いたい場合は単に
```
optim.SGD(net.paramters(), lr=0.01)
```
とすることで`Optimizer`クラスのインスタンスとしてSGDを生成し，これを使って次のように簡単に重みの更新を行うことができます：

In [9]:
import torch.optim as optim

# optimizerを定義する
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 訓練のループの中で以下の処理を行う
optimizer.zero_grad()   # 各層の重みをゼロで初期化する
output = net(input)   # 入力からモデルを通して出力を計算
loss = criterion(output, target)    # 損失を計算
loss.backward()   # バックプロパゲーション
optimizer.step()    # 重みの更新をおおなう

### 注意
`optimizer.zero_grad()`を実行して手動で勾配をゼロにリセットする必要があることに注意してください．バックプロパゲーションの項で扱ったように，`grad`には`.backward`メソッドが実行されるたびに勾配が積算されるからです．

# 課題
定義した`MyAlexNet()`クラスを参考にAlexNetを再現してください．下図（右）にAlexNetの構造の詳細を示します．左は参考のために載せたLeNetの構造の詳細です．（図は[Dive into Deep Learning](http://d2l.ai/)より引用）<br>

<img src='http://d2l.ai/_images/alexnet.svg'>

【**注意**】
- strideが指定されていないところは`stride=1`で実装してください．
- 活性化関数には`nn.ReLU`を使用してください．
- denseでは`nn.Linear`を利用してください．
- 一番右の1000はクラス数なので解きたい課題に合わせて変更できるように__init__の引数`num_classes`として持っておくと便利です．


<details>
<summary>
課題：ヒント
</summary>
3回目のmaxpoolingのあと，Full connectionするためにはtensorのshapeを(batch数, 256*6*6)のベクトルに変えた後に(batch数, 4096)への線型層にかけます<br>
Net()を参考に変えてください

</details>

In [23]:
class MyAlexNet(nn.Module):
  def __init__(self, num_classes=1000):
    super(MyAlexNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(256 * 6 * 6, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, num_classes)
 
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = F.max_pool2d(F.relu(self.conv5(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
 
  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

余談ですが，自分が実装しているモデルの中間出力のshapeを知りたいときやパラメータの数を知りたいとき，`torchsummary`を使うと便利です．課題を解く際に使ってみてください

In [11]:
from torchsummary import summary

In [24]:
mynet = MyAlexNet(1000)
mynet.to("cuda:0")
summary(mynet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
            Conv2d-2          [-1, 256, 27, 27]         614,656
            Conv2d-3          [-1, 384, 13, 13]         885,120
            Conv2d-4          [-1, 384, 13, 13]       1,327,488
            Conv2d-5          [-1, 256, 13, 13]         884,992
            Linear-6                 [-1, 4096]      37,752,832
            Linear-7                 [-1, 4096]      16,781,312
            Linear-8                 [-1, 1000]       4,097,000
Total params: 62,378,344
Trainable params: 62,378,344
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 5.03
Params size (MB): 237.95
Estimated Total Size (MB): 243.56
----------------------------------------------------------------


In [25]:
class MyAlexNet(nn.Module):
  def __init__(self, num_classes=1000):
    super(MyAlexNet, self).__init__()
    self.num_classes = 1000

    self.relu = F.relu
    self.pool = nn.MaxPool2d(2)
    
    self.conv1 = nn.Conv2d(3, 96, 11, stride=4)
    self.conv2 = nn.Conv2d(96, 256, 5, padding=2)
    self.conv3 = nn.Conv2d(256, 384, 3, padding=1)
    self.conv4 = nn.Conv2d(384, 384, 3, padding=1)
    self.conv5 = nn.Conv2d(384, 256, 3, padding=1)

    self.fc1 = nn.Linear(256 * (3 * 2) ** 2, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, self.num_classes)
 
  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.conv3(x)
    x = self.relu(x)
    x = self.conv4(x)
    x = self.relu(x)
    x = self.conv5(x)
    x = self.relu(x)
    x = self.pool(x)
    x = x.view(-1, self.num_flat_features(x))
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    return x
 
  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features


In [26]:
mynet = MyAlexNet(1000)
mynet.to("cuda:0")
summary(mynet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          34,944
         MaxPool2d-2           [-1, 96, 27, 27]               0
            Conv2d-3          [-1, 256, 27, 27]         614,656
         MaxPool2d-4          [-1, 256, 13, 13]               0
            Conv2d-5          [-1, 384, 13, 13]         885,120
            Conv2d-6          [-1, 384, 13, 13]       1,327,488
            Conv2d-7          [-1, 256, 13, 13]         884,992
         MaxPool2d-8            [-1, 256, 6, 6]               0
            Linear-9                 [-1, 4096]      37,752,832
           Linear-10                 [-1, 4096]      16,781,312
           Linear-11                 [-1, 1000]       4,097,000
Total params: 62,378,344
Trainable params: 62,378,344
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Fo

<details>
<summary>
解答
</summary>

    class MyAlexNet(nn.Module):
      def __init__(self, num_classes=1000):
        super(MyAlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

      def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(F.relu(self.conv5(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
      def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

</details>